In [24]:
import pandas as pd
import numpy as np

def get_dataframe_setdiff2d(df_new: pd.DataFrame, 
                            df_old: pd.DataFrame, 
                            rtol=1e-03, atol=1e-05) -> pd.DataFrame:
    """Returns set difference of two pandas DataFrames"""

    union_index = np.union1d(df_new.index, df_old.index)
    union_columns = np.union1d(df_new.columns, df_old.columns)

    new = df_new.reindex(index=union_index, columns=union_columns)
    old = df_old.reindex(index=union_index, columns=union_columns)

    mask_diff = ~np.isclose(new, old, rtol, atol)

    df_bool = pd.DataFrame(mask_diff, union_index, union_columns)

    df_diff = pd.concat([new[df_bool].stack(),
                         old[df_bool].stack()], axis=1)

    df_diff.columns = ["New", "Old"]

    return df_diff

"""
type_list = np.array([['Loan Type','loan_type'],['Category','category']]) 

df = pd.read_excel('source/training_data.xlsx')

for row in type_list:
    print(row[0])

df2 = df.groupby(['GENDER','District']).size().groupby(level=0).max() 
print(df2)

xdf = df['Grade'].unique().tolist()
print(np.mean(xdf,axis=0))

# given
df1 = pd.DataFrame({'A':[2,1,2],'C':[2,1,2]})
df2 = pd.DataFrame({'A':[1,1],'B':[1,1]})

diff = get_dataframe_setdiff2d(df1, df2)
print("diff:\n", diff, "\n")
"""

lowercase = lambda x: str(x).lower()
pay_df = pd.read_excel('source/pl_paymenet_date_snapshot_.xlsx',sheet_name = 'Sheet1')    
pay_df.rename(lowercase, axis='columns', inplace=True)

pay_df['snapshot_createdate'] = pd.to_datetime(pay_df['snapshot_createdate'])
exp_df = pay_df['snapshot_createdate'].sort_values(ascending=False).unique().tolist()

sel_date = pd.to_datetime('2022-11-30')

today_df = pay_df[pay_df['snapshot_createdate'] == sel_date]
today_df=today_df.assign(status='')

yes_date = pd.to_datetime(sel_date) - pd.DateOffset(1)
yes_df = pay_df[pay_df['snapshot_createdate'] == yes_date]
yes_df=yes_df.assign(status='')

#The records are find in yesterday's, but not found today's
filter1 = yes_df[~yes_df.gidpy.isin(today_df.gidpy)]
filter1['status'] ='OLD'

#The records are find in today's, but not found yesterday's
filter2 = today_df[~today_df.gidpy.isin(yes_df.gidpy)]
filter2['status'] ='NEW'
today_df.update(filter2)

filter3 = yes_df[yes_df.gidpy.isin(today_df.gidpy)]
filter3.drop(['snapshot_key', 'snapshot_createdate'], axis=1)
filter3 = filter3.drop(['snapshot_key', 'snapshot_createdate'], axis=1)

filter4 = today_df[today_df.gidpy.isin(yes_df.gidpy)]
filter4 = filter4.drop(['snapshot_key', 'snapshot_createdate'], axis=1)

comparion = filter3.reset_index(drop=True).compare(filter4.reset_index(drop=True))

#The records are find in today's and yesterday's but values not match
filter5 = today_df.iloc[comparion.index]
filter5['status'] = 'UPDATE'
today_df.update(filter5)

today_df = today_df.append(filter1)
today_df = today_df.drop(['snapshot_key', 'snapshot_createdate','gidpy','gidcq'], axis=1)

/var/folders/qq/d2bhsq8s6dv_pmg3j8k0vb5r0000gn/T/ipykernel_24259/3620028810.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter1['status'] ='OLD'
/var/folders/qq/d2bhsq8s6dv_pmg3j8k0vb5r0000gn/T/ipykernel_24259/3620028810.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter2['status'] ='NEW'
/var/folders/qq/d2bhsq8s6dv_pmg3j8k0vb5r0000gn/T/ipykernel_24259/3620028810.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co